In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import pandas as pd
import numpy as np
import csv
from lxml import html
import requests
from langdetect import detect
import math
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import json
from collections import Counter

# Question 2

## Preprocessing

In [ ]:
def clean_plot(plot):
    """ plot -> string type"""
    """ return processed plot -> string """
    
    #tokenisation
    word_tokens = word_tokenize(plot) 
    # stopwords
    stop_words = set(stopwords.words('english'))
    filter_stopsword = [w for w in word_tokens if not w in stop_words]
    # punctuations
    filter_punc = [w for w in filter_stopsword if w.isalnum()]
    # stemming
    ps = PorterStemmer() 
    filter_stem = [ps.stem(w) for w in filter_punc]
    # single string
    clean_plot = " ".join(filter_stem)
    
    return clean_plot

In [ ]:
# We do the preprocessing and the creation of the vocabulary in one go
word_list = []

# start looping over the tsv files
n = 26885+1 # the total nb of articles we have
for x in range(1,n):
    # article_
    data = pd.read_csv("html_tsv_test_"+str(x)+".tsv", sep = "\t", encoding = "utf-8")
    plot = data['Plot'].item()
    
    # replace the old plot with the new one
    data["Plot"] = data["Plot"].replace(data['Plot'].item(),clean_plot(plot))
    
    # ideally we will save over the old file
    data.to_csv("article_"+str(x)+".tsv", sep = '\t')
    
    # THIS PART TO CREATE THE VOCABULARY
    word_list.append(clean_plot(plot).split())

# the Vocabulary is a list of list, we flatten
flat_vocabulary = [item for sublist in word_list for item in sublist]
# putting the elements in a dico will eliminate the duplicates
vocabulary = dict.fromkeys(flat_vocabulary)

# We get the "integers" we are asked
i = 0
for k,v in vocabulary.items():
    vocabulary[k] = i
    i = i +1


In [ ]:
path = "C:/Users/Guillaume/Desktop/ADM_HM3"
os.chdir(path)
with open("vocabulary.json","w",encoding="utf-8") as outfile:
    outfile.write(json.dumps(vocabulary))

## 2.1 Conjunctive Query

### 2.1.1 Create your index

In [ ]:
json_file = open("vocabulary.json", "r")
vocabulary = json.load(json_file)

In [ ]:
# CVS make it faster than opening and closing each time
# changed to the correct CSV file 
df = pd.read_csv(r"articles_title_plot_url.csv", nrows=10000)

df["Tokens"] = df['Plot'].apply(lambda x: word_tokenize(clean_plot(x)))

In [ ]:
empty_voca={}
# We create a new vocabulary but without the integers and instead replace by a list we will fill
# We end up with something "could : []"
# We need to replace could by an int and fill the list with the articles
for k,v in vocabulary.items():
    empty_voca.update({k:[]})

# We use the index generated by the enumerate function
# index + 1 = article number
# So we have something like: "could : [1,25, 25,100,....]"
# We could get rid of the duplicates, they are the words that appear multiple times in a plot -> but not now
# We need to replace "could" with an integer. -> we will use the one that were in the vocabulary
for index, tokens in enumerate(df["Tokens"]):
    for token in tokens:
        empty_voca[token].append("article_"+str(index+1))

values = [x for x in empty_voca.values()]
index_1 = {v: k for k, v in vocabulary.items()}

i = 0
for k,v in index_1.items():
    index_1.update({k:values[i]})
    i += 1
index_1

In [ ]:
def query_input_1(query):
    
    # we clean the input
    query = clean_plot(query)
    # we split each word of the query
    query_words = query.split()

    # We map the word of the query to their integers.
    word_to_int = []
    for word in query_words:
        # will have to check if that "in" works as intended
        if word in vocabulary:
            word_to_int.append(vocabulary[word])

    # We ONLY want the articles for which ALL query words are present in the plot
    sets = []
    for y in word_to_int:
        # Does 12 in 120 return true ???
        if y in index_1:
            sets.append(set(index_1[y]))

    # We look for the intersection of articles
    sets = [*set.intersection(*sets)]
    
    return sets

In [ ]:
def get_output(articles_set):
    
    df = pd.DataFrame(columns = ["bookTitle","Plot","URL"])
    
    path = "C:/Users/Guillaume/Desktop/ADM_HM3/fuck_no_empty/"
    
    for article in articles_set:
        
        article = open(path+article+".tsv", "r")
        data = pd.read_csv(article, sep = "\t", encoding = "utf-8")

        title = data["bookTitle"].iloc[0]
        plot = data["Plot"].iloc[0]
        url = data["URL"].iloc[0]

        to_append = [title, plot, url]
        df_length = len(df)
        df.loc[df_length] = to_append
    
    return df.drop_duplicates()

In [ ]:
query = input()
get_output(query_input_1(query))

### 2.2 Conjunctive query & Ranking score

In [ ]:
# CVS make it faster than opening and closing each time
df = pd.read_csv(r"articles_title_plot_url.csv", nrows=1000)

df["Tokens"] = df['Plot'].apply(lambda x: word_tokenize(clean_plot(x)))

In [ ]:
index_3 = index_1.copy()

In [ ]:
# dfi is the count of documents where the term i is present
new_ = {k:len(set(v)) for k,v in index_3.items()}

In [ ]:
# tfi is the count of term i in document j
# we can just use a counter functions
df["Token_Counter"] = df["Tokens"].apply(lambda x : Counter(x)) 

In [ ]:
finfin = []

for index,tokens in enumerate(df["Token_Counter"]):
    for k,v in tokens.items():
        # v is the term frequency of term i in document j
        # len(df) is the count of documents
        # new_[vocabulary[k]] is first: vocabulary[k] 
        finfin.append((index, vocabulary[k], (v) * math.log(len(df)/new_[vocabulary[k]])))

In [ ]:
index_2 = {}

for k,v in vocabulary.items():
    index_2.update({v:[]})

In [ ]:
for k,v in index_2.items():
    for x in finfin:
        if x[1] == k:
            v.append((x[0]+1,x[2]))

In [446]:
def word_to_int(query):
    
    query = clean_plot(query)
    split_input = query.split()

    word_to_int = []
    for word in split_input:
        if word in vocabulary:
            word_to_int.a ppend(vocabulary[word])
    
    return word_to_int

In [447]:
def query_input_2(query):

    sets = []
    for y in word_to_int(query):
        if y in index_2:
            sets.append(index_2[y])

    new_sets = []
    for x in sets:
        list_articles = []
        for y in x:
            list_articles.append(y[0])
        new_sets.append(set(list_articles))

    sets = [*set.intersection(*new_sets)]

    return sets

In [448]:
def cossim(article, query):
    
    A = [x[2] for x in finfin if x[0] == article-1]
    A_int_words = [x[1] for x in finfin if x[0] == article-1]

    indexes = []
    for i in word_to_int(query):
        A_index = A_int_words.index(i)
        indexes.append(A_index)

    B = np.zeros(len(A))

    for j in indexes:
        B[j] = A[j]

    A = np.array(A)
    len_a_tfidf = np.linalg.norm(A)
    len_b_tfidf = np.linalg.norm(B)

    cossim = np.dot(A,B)/(len_a_tfidf*len_b_tfidf)
    
    return cossim

In [459]:
def get_output_score(articles_set, query):
    
    df = pd.DataFrame(columns = ["bookTitle","Plot","URL","Similarity"])

    path = "C:/Users/Guillaume/Desktop/ADM_HM3/fuck_no_empty/"
    
    for article in articles_set:
        
        article_doc = open(path+"article_"+str(article)+".tsv", "r", encoding="utf-8")
        data = pd.read_csv(article_doc, sep = "\t", encoding = "utf-8")

        title = data["bookTitle"].iloc[0]
        plot = data["Plot"].iloc[0]
        url = data["URL"].iloc[0]
        Sim_score = float(cossim(article, query))

        to_append = [title, plot, url, Sim_score]
        df_length = len(df)
        df.loc[df_length] = to_append
    
    return df.drop_duplicates().sort_values(by="Similarity",ascending=False)

In [461]:
query = input()
get_output_score(query_input_2(query), query).head(10)

hitler


,bookTitle,Plot,URL,Similarity
2,The Power of One,"In 1939, as Hitler casts his enormous, cruel s...",https://www.goodreads.com/book/show/122.The_Po...,0.188018
1,The Complete Maus,Combined for the first time here are Maus I: A...,https://www.goodreads.com/book/show/15195.The_...,0.171166
3,The Bronze Horseman,"The golden skies, the translucent twilight, th...",https://www.goodreads.com/book/show/41014505-t...,0.116016
0,Seabiscuit: An American Legend,There's an alternate cover edition hereSeabisc...,https://www.goodreads.com/book/show/110737.Sea...,0.092499
4,Corelli's Mandolin,Captain Corelli’s Mandolin is set in the early...,https://www.goodreads.com/book/show/3388.Corel...,0.069524
